In [25]:
import numpy as np
import pandas as pd
import pickle
import re
import keras
from keras.layers import Embedding

In [2]:
# to communicate with google spreadsheet...
import gspread
from gspread_dataframe import get_as_dataframe
from gspread_dataframe import set_with_dataframe
from google.oauth2 import service_account # based on google-auth library
import sddk

s = sddk.cloudSession("sciencedata.dk")
# establish connection with gogglesheets...
file_data = s.read_file("https://sciencedata.dk/files/ServiceAccountsKey.json", "dict") # or load it from a local storage: json.load(open("../../ServiceAccountsKey.json", "r"))
credentials = service_account.Credentials.from_service_account_info(file_data)
gc = gspread.Client(auth=credentials.with_scopes(['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']))
PIPA_data = gc.open_by_url("https://docs.google.com/spreadsheets/d/1rV4t0_UV_wcx--UAHVwkqB8Wa_5n9mnpV05yGG1OHqk/edit?usp=sharing")

endpoint variable has been configured to: https://sciencedata.dk/files/


In [3]:
cgl = pd.read_json("../data/large_data/cgl.json")
cgl.head()

,filename,author,title,wordcount,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,...,lemmata,lemmata_wordcount,subcorpus,lemmata_repl,lemmatized_sentences_repl,count_πόνο*,count_ὀδύν*,count_ἄλγ*,count_λύπ*,conc_lype
1,tlg0003.tlg001.perseus-grc2.xml,Thucydides,The Peloponnesian War,150118,tlg0003,tlg0003.tlg001,5 B.C.,-4.5,{'-4.5': 1},-4.5,...,"[θουκυδίδης, Ἀθηναῖος, συγγράφω, πόλεμος, Πελο...",71863,None,"[θουκυδίδης, Ἀθηναῖος, συγγράφω, πόλεμος, Πελο...","[[θουκυδίδης, Ἀθηναῖος, συγγράφω, πόλεμος, Πελ...",31,0,6,25,"[[μέγας, κινδυνεύοντας, δέχομαι, ἀείμνηστος, μ..."
6,tlg0006.tlg001.perseus-grc2.xml,Euripides,Cyclops,4141,tlg0006,tlg0006.tlg001,5 B.C.,-4.5,{'-4.5': 1},-4.5,...,"[Βρόμιος, ἔχω, πόνος, χὥτʼ, ἥβη, ἐμός, εὐσθενέ...",2535,None,"[Βρόμιος, ἔχω, πόνο*, χὥτʼ, ἥβη, ἐμός, εὐσθενέ...","[[Βρόμιος, ἔχω, πόνο*, χὥτʼ, ἥβη, ἐμός, εὐσθεν...",7,0,0,1,"[[βοτόν, οὔτις, θύω, θεός, μέγας, γαστήρ, δαίμ..."
7,tlg0006.tlg004.perseus-grc2.xml,Euripides,Ἡρακλεῖδαι,6272,tlg0006,tlg0006.tlg004,5 B.C.,-4.5,{'-4.5': 1},-4.5,...,"[ποτός, εἰμί, οὗτος, δεδογμένον, δίκαιος, φύω,...",3545,None,"[ποτός, εἰμί, οὗτος, δεδογμένον, δίκαιος, φύω,...","[[ποτός, εἰμί, οὗτος, δεδογμένον], [δίκαιος, φ...",11,0,1,2,"[[εἰμί, πολύς, χαίρω, δυσφημέω, ἅζομαι, θέα, σ..."
8,tlg0006.tlg005.perseus-grc2.xml,Euripides,Ἱππόλυτος,8257,tlg0006,tlg0006.tlg005,5 B.C.,-4.5,{'-4.5': 1},-4.5,...,"[πολύς, βροτός, ἀνώνυμος, θεά, καλέω, Κύπρις, ...",4898,None,"[πολύς, βροτός, ἀνώνυμος, θεά, καλέω, Κύπρις, ...","[[πολύς, βροτός, ἀνώνυμος, θεά, καλέω, Κύπρις,...",8,3,8,8,"[[κοιτάζω, λέχος, σός, ναυβάτης, τὶς, πλέω, κρ..."
9,tlg0006.tlg006.perseus-grc2.xml,Euripides,Ἀνδρομάχη,7397,tlg0006,tlg0006.tlg006,5 B.C.,-4.5,{'-4.5': 1},-4.5,...,"[Ἀσιανός, γῆ, σχῆμα, θηβαία, πόλις, ἑδνόω, πολ...",4420,None,"[Ἀσιανός, γῆ, σχῆμα, θηβαία, πόλις, ἑδνόω, πολ...","[[Ἀσιανός, γῆ, σχῆμα, θηβαία, πόλις, ἑδνόω, πο...",5,0,7,4,"[[Τροία, πράσσω, μηδείς, ὅδε, αὐχέω, πράσσω, δ..."


In [4]:
cgl_embeddings = pickle.load(open("../data/large_data/cgl_embeddings.pkl", "rb"))

In [5]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [6]:
len(range(1, len(cgl_embeddings[1])+1))

5000

In [7]:
word_index = dict(zip(cgl_embeddings[1],range(1, len(cgl_embeddings[1])+1)))
list(word_index.items())[:10]

[('εἰμί', 1),
 ('οὗτος', 2),
 ('λέγω', 3),
 ('ἔχω', 4),
 ('γίγνομαι', 5),
 ('πολύς', 6),
 ('ἄλλος', 7),
 ('αὐτός', 8),
 ('πᾶς', 9),
 ('ποιέω', 10)]

In [41]:
# we will generate the concordances again, now focusing on words in the embeddings only
def get_concordances(wordlist, keyword, window, vocabulary=None):
    half = int(window / 2)
    if vocabulary != None:
        wordlist = [w for w in wordlist if w in vocabulary]
    keyword_indices = [el[0] for el in enumerate(wordlist) if el[1]==keyword]
    concordances = [wordlist[i-half:i+half+1] for i in keyword_indices]
    #concordances = [c for c in concordances if len(c)==window]
    return concordances

In [42]:
cgl["conc_lype"] = cgl["lemmata_repl"].apply(lambda x: get_concordances(x, "λύπ*", 31, vocabulary=cgl_embeddings[1]))

In [43]:
cgl = cgl[cgl["author_id"]=="tlg0086"].copy()
cgl

,filename,author,title,wordcount,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,...,lemmata_wordcount,subcorpus,lemmata_repl,lemmatized_sentences_repl,count_πόνο*,count_ὀδύν*,count_ἄλγ*,count_λύπ*,conc_lype,sentences
685,tlg0086.tlg001.1st1K-grc2.xml,Aristotle,Aristotelis Analytica Priora et Posteriora,59614,tlg0086,tlg0086.tlg001,4 B.C.,-3.5,{'-3.5': 1},-3.5,...,24680,c_aristotelicum,"[πρῶτος, λέγω, εἰμί, σκέψις, ἀπόδειξις, ἐπιστή...","[[πρῶτος, λέγω, εἰμί, σκέψις, ἀπόδειξις, ἐπιστ...",0,0,1,0,[],[Πρῶτον εἰπεῖν περὶ τί καὶ τίνος ἐστὶν ἡ σκέψι...
686,tlg0086.tlg002.1st1K-grc2.xml,Aristotle,De anima,20912,tlg0086,tlg0086.tlg002,4 B.C.,-3.5,{'-3.5': 1},-3.5,...,9395,c_aristotelicum,"[καλός, τίμιος, εἶδος, ὑπολαμβάνω, ἕτερος, ἕτε...","[[καλός, τίμιος, εἶδος, ὑπολαμβάνω, ἕτερος, ἕτ...",0,0,0,12,"[[κινέω, εἰμί, κινέω, ψυχή, οἷος, κινέω, τόπος...",[Τῶν καλῶν καὶ τιμίων τὴν εἴδησιν ὑπολαμβάνοντ...
687,tlg0086.tlg003.perseus-grc1.xml,Aristotle,Athenian Constitution,16459,tlg0086,tlg0086.tlg003,4 B.C.,-3.5,{'-3.5': 1},-3.5,...,8088,None,"[Ἀθηναῖος, ἀρχή, χράω, βασίλειος, συνοικησάντο...","[[Ἀθηναῖος, ἀρχή, χράω, βασίλειος], [συνοικησά...",0,2,1,0,[],"[Ἀθηναῖοι τὸ μὲν ἐξ ἀρχῆς ἐχρῶντο βασιλείᾳ., σ..."
688,tlg0086.tlg005.1st1K-grc1.xml,Aristotle,De caelo,30794,tlg0086,tlg0086.tlg005,4 B.C.,-3.5,{'-3.5': 1},-3.5,...,13437,c_aristotelicum,"[φύσις, ἐπιστήμη, πλεῖστος, φαίνω, σῶμα, μέγεθ...","[[φύσις, ἐπιστήμη, πλεῖστος, φαίνω, σῶμα, μέγε...",1,0,0,0,[],[Ἡ περὶ φύσεως ἐπιστήμη σχεδὸν ἡ πλείστη φαίνε...
689,tlg0086.tlg006.1st1K-grc1.xml,Aristotle,Categoriae,10316,tlg0086,tlg0086.tlg006,4 B.C.,-3.5,{'-3.5': 1},-3.5,...,4678,c_aristotelicum,"[ὁμώνυμος, λέγω, ὄνομα, μόνος, κοινός, ὄνομα, ...","[[ὁμώνυμος, λέγω, ὄνομα, μόνος, κοινός, ὄνομα,...",0,0,0,2,"[[φυσικός, ἄλλος, σύμπτωμα, γίγνομαι, ἀκίνητος...","[ὉΜΩΝΥΜΑ λέγεται ὧν ὄνομα μόνον κοινόν, ὁ δὲ κ..."
690,tlg0086.tlg008.1st1K-grc1.xml,Aristotle,De divinatione per somnum,1194,tlg0086,tlg0086.tlg008,4 B.C.,-3.5,{'-3.5': 1},-3.5,...,549,c_aristotelicum,"[μαντικός, ὕπνος, γίγνομαι, λέγω, συμβαίνω, ἐν...","[[μαντικός, ὕπνος, γίγνομαι, λέγω, συμβαίνω, ἐ...",0,0,0,0,[],[Περὶ δὲ τῆς μαντικῆς τῆς ἐν τοῖς ὕπνοις γινομ...
691,tlg0086.tlg009.perseus-grc1.xml,Aristotle,Eudemian Ethics (Greek). Machine readable text,26345,tlg0086,tlg0086.tlg009,4 B.C.,-3.5,{'-3.5': 1},-3.5,...,11564,c_aristotelicum,"[δῆλος, θεός, ἑαυτοῦ, γνώμη, ἀποφηνάμενος, συγ...","[[δῆλος, θεός, ἑαυτοῦ, γνώμη, ἀποφηνάμενος, συ...",7,1,4,76,"[[εἰμί, αἵρεσις, δίδωμι, γίγνομαι, βίος, ζῶ, π...",[ὁ μὲν ἐν Δήλῳ παρὰ τῷ θεῷ τὴν αὑτοῦ γνώμην ἀπ...
692,tlg0086.tlg010.perseus-grc1.xml,Aristotle,Nicomachean Ethics,56620,tlg0086,tlg0086.tlg010,4 B.C.,-3.5,{'-3.5': 1},-3.5,...,25082,c_aristotelicum,"[τέχνη, πᾶς, μέθοδος, πρᾶξις, προαίρεσις, ἀγαθ...","[[τέχνη, πᾶς, μέθοδος, πρᾶξις, προαίρεσις, ἀγα...",11,0,6,153,"[[ῥοπή, ζωή, μέγας, πολύς, γίγνομαι, μακάριος,...","[πᾶσα τέχνη καὶ πᾶσα μέθοδος, ὁμοίως δὲ πρᾶξίς..."
693,tlg0086.tlg013.1st1K-grc2.xml,Aristotle,De generatione et corruptione,16914,tlg0086,tlg0086.tlg013,4 B.C.,-3.5,{'-3.5': 1},-3.5,...,7430,None,"[γένεσις, φθορά, φύσις, γίγνομαι, φθειρομένων,...","[[γένεσις, φθορά, φύσις, γίγνομαι, φθειρομένων...",0,0,0,1,"[[ἅπτω, κινέω, κινούμενα, ὁμογενής, ἀνάγκη, δο...",[Περὶ δὲ γενέσεως καὶ φθορᾶς τῶν φύσει γινομέν...
694,tlg0086.tlg014.1st1K-grc1.xml,Aristotle,Historia animalium,93923,tlg0086,tlg0086.tlg014,4 B.C.,-3.5,{'-3.5': 1},-3.5,...,43967,c_aristotelicum,"[ζῷον, μόριον, εἰμί, ἀσύνθετος, ὅσος, διαιρέω,...","[[ζῷον, μόριον, εἰμί, ἀσύνθετος, ὅσος, διαιρέω...",40,0,12,5,"[[κεφαλή, συνόχωκα, αὐτός, οὗτος, τείνω, σφαγή...",[ΤΩΝ ἐν τοῖς ζῴοις μορίων τὰ μέν ἐστιν ἀσύνθετ...


In [44]:
ethical_ids = ["tlg0086.tlg009", "tlg0086.tlg010", "tlg0086.tlg022", "tlg0086.tlg035", "tlg0086.tlg038",]

In [46]:
v_ud = "’"
v_agt = "ʼ"

def get_sentences(string):
    splits = re.split(r"(\.|\·|\:|\;|\?|\!)" , string)
    sentences = []
    for n in range(0, len(splits), 2):
        try:
            sentence = splits[n]+splits[n+1]
            sentence = "".join([c for c in sentence if re.match("[\w|\s|\.|\,|\·|\:|\;|\?|\!]", c)])
            sentence = re.sub("^\s*", "", sentence)
            sentence = re.sub(v_ud, v_agt, sentence)
            sentences.append(sentence)
        except:
            sentence = splits[n]
            sentence = re.sub("^\s*", "", sentence)
            if len(sentence) > 0:
                sentences.append(sentence)
            break
    return sentences
cgl["sentences"] = cgl["clean_string"].apply(get_sentences)

In [51]:
cgl["sentences"].apply(len).sum()

60282

In [52]:
cgl["lemmatized_sentences"].apply(len).sum()

60282

perfect, we have the same number of sentences as lemmatized sentences

In [50]:
cgl["lemmatized_sentences_repl"].apply(lambda x: sum([sent.count("λύπ*") for sent in x])).sum()

414

In [49]:
cgl["conc_lype"].apply(lambda x: len(x)).sum()

414

perfect, we have the same number of lype instances matched in concordances as in lemmatized sentences...

In [37]:
[["dswd", "ewewe"]] * 1

[['dswd', 'ewewe']]

In [53]:
cgl["lemmatized_sentences_repl"].apply(lambda x: [sent.count("λύπ*") for sent in x])


685    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
686    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
687    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
688    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
689    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
690    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
691    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
692    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
693    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
694    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
695    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
696    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
697    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
698    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
699    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
700    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
701    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
702    [0, 0, 0, 0, 0, 0, 0, 0,

In [62]:
cgl["lemmatized_sentences_repl"].apply(lambda x: [(i,j.count("λύπ*")) for i,j in enumerate(x) if "λύπ*" in j]).tolist()[1]

[(257, 1),
 (261, 1),
 (305, 1),
 (477, 1),
 (499, 2),
 (783, 1),
 (1136, 1),
 (1137, 1),
 (1155, 1),
 (1245, 1),
 (1316, 1)]

In [66]:
pd.Series(["sent1", "sent2", "sent3", "sent4"])[[1,3]]

1    sent2
3    sent4
dtype: object

In [67]:
["sent1"] * 3

'sent1sent1sent1'

In [82]:
def sentences_with_lype(work_row):
    inds_counts_tups = [(i,j.count("λύπ*")) for i,j in enumerate(work_row["lemmatized_sentences_repl"]) if "λύπ*" in j]
    sentences = work_row["sentences"]
    lype_sentences = []
    for tup in inds_counts_tups:
        lype_sentences.extend([sentences[tup[0]]] * tup[1])
    return lype_sentences

In [83]:
cgl["sentences_with_lype"] = cgl.apply(sentences_with_lype, axis=1)

In [86]:
cgl["sentences_with_lype"].apply(len).sum()

414

In [87]:
cgl["conc_lype"].apply(len).sum()


414

In [88]:
cgl["sentences_with_lype"].tolist()[1]

['φαμὲν γὰρ τὴν ψυχὴν λυπεῖσθαι χαίρειν, θαρρεῖν φοβεῖσθαι, ἔτι δὲ ὀργίζεσθαί τε καὶ αἰσθάνεσθαι καὶ διανοεῖσθαι·',
 'εἰ γὰρ καὶ ὅτι μάλιστα τὸ λυπεῖσθαι ἢ χαίρειν ἢ διανοεῖσθαι κινήσεις εἰσί, καὶ ἕκαστον κινεῖσθαι τούτων, τὸ δὲ κινεῖσθαί ἐστιν ὑπὸ τῆς ψυχῆς, οἷον τὸ ὀργίζεσθαι ἢ φοβεῖσθαι τὸ τὴν καρδίαν ὡδὶ κινεῖσθαι, τὸ δὲ διανοεῖσθαι ἢ τὸ τοῦτο ἴσως ἢ ἕτερόν τι, τούτων δὲ συμβαίνει τὰ μὲν κατὰ φορὰν τινῶν κινουμένων, τὰ δὲ κατʼ ἀλλοίωσιν ποῖα δὲ καὶ πῶς, ἕτερός ἐστι λόγος τὸ δὴ λέγειν ὀργίζεσθαι τὴν ψυχὴν ὅμοιον κἂν εἴ τις λέγοι τὴν ψυχὴν ὑφαίνειν ἢ οἰκοδομεῖν·',
 'δῆλον δʼ εἴ τις ἐπιχειρήσειεν ἐκ τοῦ λόγου τούτου τὰ πάθη καὶ τὰ ἔργα τῆς ψυχῆς ἀποδιδόναι, οἷον λογισμούς, αἰσθήσεις, ἡδονάς, λύπας, ὅσα ἄλλα τοιαῦτα·',
 'ὅπου μὲν γὰρ αἴσθησις, καὶ λύπη τε καὶ ἡδονή, ὅπου δὲ ταῦτα, ἐξ ἀνάγκης καὶ ἐπιθυμία.',
 'ὄρεξις μὲν γὰρ ἐπιθυμία καὶ θυμὸς καὶ βούλησις, τὰ δὲ ζῷα πάντʼ ἔχουσι μίαν γε τῶν αἰσθήσεων, τὴν ἁφήν, ᾧ δʼ αἴσθησις ὑπάρχει, τούτῳ ἡδονή τε καὶ λύπη καὶ τὸ ἡδύ τε καὶ λυπηρόν, ο

In [97]:
ethical_ids = ["tlg0086.tlg009", "tlg0086.tlg010", "tlg0086.tlg022", "tlg0086.tlg035", "tlg0086.tlg038",]

In [94]:
lype_doc_ids = [el for ellist in cgl.apply(lambda row: [row["doc_id"] for conc in row["conc_lype"]], axis=1) for el in ellist]
lype_doc_titles = [el for ellist in cgl.apply(lambda row: [row["title"] for conc in row["conc_lype"]], axis=1)  for el in ellist]
lype_concs = [el for ellist in cgl["conc_lype"] for el in ellist]
lype_sents = [el for ellist in cgl["sentences_with_lype"] for el in ellist]

In [95]:
# labels
labels = [1 if doc_id in ethical_ids else 0 for doc_id in lype_doc_ids]

In [99]:
lype_concs_vectorized = np.array([[word_index[w] for w in conc] for conc in lype_concs])

In [100]:
lype_concs_vectorized[:5]

array([list([103, 1, 103, 78, 20, 103, 129, 8, 1491, 636, 22, 17, 1776, 11, 78, 208, 268, 832, 309, 1040, 283, 700, 2, 9, 126, 1, 16, 45, 8, 103, 1]),
       list([832, 309, 1040, 283, 700, 2, 9, 126, 1, 16, 45, 8, 103, 1, 110, 208, 268, 700, 126, 1, 28, 103, 103, 78, 20, 1040, 309, 553, 103, 700, 2]),
       list([17, 1, 1022, 89, 22, 312, 12, 227, 75, 78, 156, 20, 864, 263, 144, 208, 59, 7, 17, 413, 48, 2728, 255, 68, 307, 398, 78, 552, 103, 67, 36]),
       list([25, 308, 78, 85, 2522, 263, 159, 81, 4, 126, 129, 263, 1791, 1557, 263, 208, 144, 52, 431, 209, 2598, 79, 15, 112, 93, 78, 65, 25, 1, 34, 162]),
       list([1971, 1557, 431, 612, 1636, 155, 9, 4, 61, 263, 1696, 263, 46, 2, 144, 208, 195, 208, 431, 195, 1557, 2, 218, 263, 4, 218, 263, 351, 187, 171, 282])],
      dtype=object)

In [169]:
indices = np.arange(len(concs_tups))
np.random.seed(0)
np.random.shuffle(indices)

In [214]:
data = np.array([conc[1] for conc in concs_tups])
labels = []
for conc in concs_tups:
    if conc[0] in ethical_ids:
        labels.append(1)
    else:
        labels.append(0)
labels = np.array(labels)

In [215]:
data = data[indices]
#data = np.array([conc[10:-10] for conc in data])
labels = labels[indices]

In [216]:
labels

array([1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,

In [217]:
labels

array([1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,

In [218]:
cgl_embeddings[3]

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
εἰμί,40.921943,23.975110,6.083791,1.724597,0.787708,1.395476,10.085121,-3.512090,-2.466157,-0.479203,...,0.135525,0.116194,0.038779,0.028518,0.048852,0.063003,-0.006917,-0.285075,0.083432,0.050213
οὗτος,40.370998,23.444483,5.179671,1.611451,0.341264,1.735556,9.830402,-3.353345,-2.345492,-0.181414,...,0.139695,0.078060,0.127660,0.069392,0.060385,0.249957,0.070054,-0.194425,-0.004014,0.040241
λέγω,39.001855,22.120811,3.711950,0.918392,1.452316,0.077037,9.391213,-2.759150,-2.398183,0.214704,...,-0.031307,0.068023,0.000057,0.069156,0.111072,0.038494,0.002585,-0.019338,0.269553,-0.130885
ἔχω,39.804302,23.024422,6.199601,2.275942,0.625015,1.243543,9.147309,-3.485462,-2.581793,-0.275819,...,0.024101,0.063533,0.134114,0.179058,0.043486,0.113336,-0.036198,-0.342098,0.209940,-0.073624
γίγνομαι,39.347879,22.489298,5.427730,1.608740,-0.299176,1.453300,8.521668,-3.282159,-2.755167,-0.361946,...,0.017706,0.150147,-0.100498,0.024932,-0.241511,0.180704,-0.044182,-0.225451,-0.009623,0.082624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
κύαθος,2.306894,-1.252014,1.967848,0.639823,-0.830738,1.400087,0.623265,-0.022984,1.364765,-1.015133,...,0.029445,0.035054,-0.064084,-0.022086,-0.026919,-0.029758,-0.059165,0.081717,0.070341,0.065586
σκυτεύς,2.426349,-1.921372,0.401636,-0.442950,-0.050278,0.079601,1.183014,-0.539222,0.169868,0.727299,...,-0.379041,0.069706,0.277146,0.071542,-0.010878,0.112946,-0.114023,0.315502,-0.062303,-0.036280
παραμίγνυμι,2.460930,-1.073599,2.207248,0.563545,-0.814028,1.938490,0.646300,-0.572682,2.623897,-1.114314,...,-0.005971,-0.170639,-0.140238,0.136425,0.311922,0.270307,-0.047148,-0.198601,-0.085457,0.311602
ὀφθαλμία,1.968229,-1.475017,1.460325,-0.047004,-0.329035,0.401820,0.145133,-0.578576,-0.902634,0.095910,...,-0.229633,-0.162759,-0.336879,-0.035314,0.135483,-0.145982,0.128570,-0.032265,-0.125454,-0.178760


In [219]:
training_samples = int(len(data) * 0.70)
test_samples = int(len(data) * 0.1)

x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples:-test_samples]
y_val = labels[training_samples:-test_samples]

x_test = data[-test_samples:]
y_test = labels[-test_samples:]

In [220]:
embedding_matrix = cgl_embeddings[3]
embedding_matrix

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
εἰμί,40.921943,23.975110,6.083791,1.724597,0.787708,1.395476,10.085121,-3.512090,-2.466157,-0.479203,...,0.135525,0.116194,0.038779,0.028518,0.048852,0.063003,-0.006917,-0.285075,0.083432,0.050213
οὗτος,40.370998,23.444483,5.179671,1.611451,0.341264,1.735556,9.830402,-3.353345,-2.345492,-0.181414,...,0.139695,0.078060,0.127660,0.069392,0.060385,0.249957,0.070054,-0.194425,-0.004014,0.040241
λέγω,39.001855,22.120811,3.711950,0.918392,1.452316,0.077037,9.391213,-2.759150,-2.398183,0.214704,...,-0.031307,0.068023,0.000057,0.069156,0.111072,0.038494,0.002585,-0.019338,0.269553,-0.130885
ἔχω,39.804302,23.024422,6.199601,2.275942,0.625015,1.243543,9.147309,-3.485462,-2.581793,-0.275819,...,0.024101,0.063533,0.134114,0.179058,0.043486,0.113336,-0.036198,-0.342098,0.209940,-0.073624
γίγνομαι,39.347879,22.489298,5.427730,1.608740,-0.299176,1.453300,8.521668,-3.282159,-2.755167,-0.361946,...,0.017706,0.150147,-0.100498,0.024932,-0.241511,0.180704,-0.044182,-0.225451,-0.009623,0.082624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
κύαθος,2.306894,-1.252014,1.967848,0.639823,-0.830738,1.400087,0.623265,-0.022984,1.364765,-1.015133,...,0.029445,0.035054,-0.064084,-0.022086,-0.026919,-0.029758,-0.059165,0.081717,0.070341,0.065586
σκυτεύς,2.426349,-1.921372,0.401636,-0.442950,-0.050278,0.079601,1.183014,-0.539222,0.169868,0.727299,...,-0.379041,0.069706,0.277146,0.071542,-0.010878,0.112946,-0.114023,0.315502,-0.062303,-0.036280
παραμίγνυμι,2.460930,-1.073599,2.207248,0.563545,-0.814028,1.938490,0.646300,-0.572682,2.623897,-1.114314,...,-0.005971,-0.170639,-0.140238,0.136425,0.311922,0.270307,-0.047148,-0.198601,-0.085457,0.311602
ὀφθαλμία,1.968229,-1.475017,1.460325,-0.047004,-0.329035,0.401820,0.145133,-0.578576,-0.902634,0.095910,...,-0.229633,-0.162759,-0.336879,-0.035314,0.135483,-0.145982,0.128570,-0.032265,-0.125454,-0.178760


In [221]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

model = Sequential()
model.add(Embedding(5000,  150, input_length=31))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    (None, 31, 150)           750000    
                                                                 
 flatten_13 (Flatten)        (None, 4650)              0         
                                                                 
 dense_26 (Dense)            (None, 32)                148832    
                                                                 
 dense_27 (Dense)            (None, 1)                 33        
                                                                 
Total params: 898,865
Trainable params: 898,865
Non-trainable params: 0
_________________________________________________________________


In [222]:
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

In [223]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_data=(x_val, y_val))
model.save_weights('pre_trained_glove_model.h5')

Epoch 1/10
10/10 [==============================] - 0s 8ms/step - loss: 0.8321 - acc: 0.7128 - val_loss: 0.4622 - val_acc: 0.8795
Epoch 2/10
10/10 [==============================] - 0s 2ms/step - loss: 0.1681 - acc: 0.9377 - val_loss: 0.4432 - val_acc: 0.8795
Epoch 3/10
10/10 [==============================] - 0s 2ms/step - loss: 0.0796 - acc: 0.9792 - val_loss: 0.4991 - val_acc: 0.8916
Epoch 4/10
10/10 [==============================] - 0s 3ms/step - loss: 0.0417 - acc: 0.9896 - val_loss: 0.4900 - val_acc: 0.8795
Epoch 5/10
10/10 [==============================] - 0s 2ms/step - loss: 0.0169 - acc: 1.0000 - val_loss: 0.5388 - val_acc: 0.8795
Epoch 6/10
10/10 [==============================] - 0s 3ms/step - loss: 0.0090 - acc: 1.0000 - val_loss: 0.5771 - val_acc: 0.8795
Epoch 7/10
10/10 [==============================] - 0s 2ms/step - loss: 0.0053 - acc: 1.0000 - val_loss: 0.5617 - val_acc: 0.8675
Epoch 8/10
10/10 [==============================] - 0s 3ms/step - loss: 0.0027 - acc: 1.00

In [224]:
test_predict = model.predict(x_test)

In [225]:
pred_df = pd.DataFrame(test_predict, y_test).reset_index().rename(columns={"index":"reality", 0: "model_prob"})
pred_df

,reality,model_prob
0,1,0.999414
1,1,0.999607
2,1,0.999993
3,1,0.997477
4,0,0.981670
5,1,0.904934
6,1,0.999976
7,1,0.999139
8,1,0.991690
9,1,0.999179
